In [ ]:
# default_exp trainers.MSResNet

In [ ]:
# export
import torch
from torch.utils.data import SequentialSampler, RandomSampler, DataLoader
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel, DataParallel
import torch.distributed as dist

In [ ]:
# export
class Trainer():
    def __init__(self, 
                 model, 
                 train_dataset, validation_dataset, 
                 batch_size, 
                 num_workers, 
                 world_size=1,
                 device_id=0 ):
        
        self.model = model
        self.device_id = device_id
        self.distributed = True if world_size>1 else False
        
        '''
        Parallelization stuff
        '''
        if world_size==1:
            train_sampler = RandomSampler(train_dataset, replacement=False)
            validation_sampler = SequentialSampler(validation_dataset)
            if self.device_id!='cpu':
                self.model.cuda(self.device_id)         
        else:
            # if distributed, use the distributed sampler
            train_sampler = DistributedSampler(train_dataset, 
                                               shuffle=True, 
                                               num_replicas=world_size, 
                                               rank=self.device_id,
                                              )
            validation_sampler = DistributedSampler(validation_dataset, 
                                                    shuffle=False, 
                                                    num_replicas=world_size, 
                                                    rank=self.device_id,
                                                   )
            # also, wrap the networks with DistributedDataParallel
            if self.model.G is not None:
                self.model.G.cuda(self.device_id)
                self.model.G = DistributedDataParallel(self.model.G, device_ids=[self.device_id], output_device=self.device_id)
            if self.model.D is not None:
                self.model.D.cuda(self.device_id)
                self.model.D = DistributedDataParallel(self.model.D, device_ids=[self.device_id], output_device=self.device_id)
            if self.model.P is not None:
                self.model.P.cuda(self.device_id)
                self.model.P = DistributedDataParallel(self.model.P, device_ids=[self.device_id], output_device=self.device_id)
            
        self.train_loader = DataLoader(dataset=train_dataset,
                                       batch_size=batch_size,
                                       shuffle=False,
                                       sampler=train_sampler,
                                       num_workers=num_workers,
                                       pin_memory=True,
                                       drop_last=True,
                                      )
        self.val_loader = DataLoader(dataset=validation_dataset,
                                     batch_size=batch_size,
                                     shuffle=False,
                                     sampler=validation_sampler,
                                     num_workers=num_workers,
                                     pin_memory=True,
                                     drop_last=False,
                                    )
        self.count = 0
        self.synchronized = False
        
    def all_reduce(self, epoch=None):
        # synchronize loss for distributed GPU processes

        if epoch is None:
            epoch = self.epoch

        def _reduce_value(value, ReduceOp=dist.ReduceOp.SUM):
            value_tensor = torch.Tensor([value]).to(self.device_id, non_blocking=True)
            dist.all_reduce(value_tensor, ReduceOp, async_op=False)
            value = value_tensor.item()
            del value_tensor

            return value

        dist.barrier()
        if self.count > 0: 
            self.count = _reduce_value(self.count, dist.ReduceOp.SUM)

            
            '''
            # this part is TODO
            for loss_type in self.loss_stat[self.mode]:
                self.loss_stat[self.mode][loss_type][epoch] = _reduce_value(
                    self.loss_stat[self.mode][loss_type][epoch],
                    dist.ReduceOp.SUM)
                '''
        self.synchronized = True

        return
   
    def train(self, epoch):
        self.epoch = epoch
        for sample in self.train_loader:
            if self.device_id != 'cpu':
                input_ = [data.cuda(self.device_id) for data in sample['input']]
                target = [data.cuda(self.device_id) for data in sample['target']]
            else:
                input_ = [data for data in sample['input']]
                target = [data for data in sample['target']]

            self.model.G_optimizer.zero_grad()
            self.model.D_optimizer.zero_grad()
            
            # do I need this synch flag?
            self.synchronized = False
            
            output = self.model.G(input_)
            loss_g_rec = 0
            for scaled_output, scaled_target in zip(output, target):
                loss_g_rec += self.model.reconstruction_loss(scaled_output, scaled_target)

            # update discriminator
            high_res_output = output[-1]
            high_res_target = target[-1]
            d_fake = self.model.D(high_res_output.detach())
            d_real = self.model.D(high_res_target)    
            label_fake = torch.zeros_like(d_fake)
            label_real = torch.ones_like(d_real)    
            loss_d = self.model.adversarial_loss(d_fake, label_fake) + self.model.adversarial_loss(d_real, label_real)
            loss_d.backward()
            self.model.D_optimizer.step()

            # update generator
            # do I need a barrier here or not?
            if self.distributed:
                dist.barrier()
            d_fake_with_gradient = self.model.D(high_res_output)
            loss_g_adversarial = self.model.adversarial_loss(d_fake_with_gradient, label_real)
            loss_g = loss_g_adversarial + loss_g_rec
            loss_g.backward()
            self.model.G_optimizer.step()
            
            self.count+=1
        
        if self.distributed:
            dist.barrier()
            if not self.synchronized:
                self.all_reduce()
        self.count = 0
        if self.device_id == 0 and epoch % 5 == 0:
            self.model.save("epoch_{}.pth".format(epoch, self.device_id))

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_model.ipynb.
Converted 02_architecture_common.ipynb.
Converted 03_architecture_MSResNet.ipynb.
Converted 04_dataset_common.ipynb.
Converted 05_dataset_MSResNet.ipynb.
Converted 06_trainer_MSResNet.ipynb.
Converted 99_diffaugment.ipynb.
Converted Tutorial.ipynb.
Converted lightning_trial.ipynb.
Converted trials.ipynb.
